# Test de entramiento 

### En este notebook encontraremos Una serie de entrenamientos y validaciones de nuestro modelo para concluir con el mejor posible, utilizaremos primero todas las variables ya que si bien conseguimos algunas variables que más se correlacionan con el G3 al no conocer bien los datos y el modelo exacto del nogico en este caso los colegios no sabemos si pueden haber o no variables que sean influyentes al modelo

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split #Esta parte nos sirve para la separación de nuestros datos para que el modelo no memorice
from sklearn.preprocessing import StandardScaler #Estandarizar nuestros valores con mean = 0 , std = 1 para no confusiones del modelo
from sklearn.linear_model import LinearRegression #Modelo de regresion lineal 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score #Métricas para evaluar nuestro modelo

df = pd.read_csv('../data/student-por.csv', sep=';') #Cargamos los datos 
df.head(5) #Revisamos 5 primeros datos

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


In [28]:
numericas = df.select_dtypes(include=['int64', 'float64']).columns.to_list()
numericas

['age',
 'Medu',
 'Fedu',
 'traveltime',
 'studytime',
 'failures',
 'famrel',
 'freetime',
 'goout',
 'Dalc',
 'Walc',
 'health',
 'absences',
 'G1',
 'G2',
 'G3']

In [29]:
#Creamos una copia del dataframe para convertir las variables categoricas a numeros para poder usarlos en el entrenamiento
df_copy = df.copy()

#Convertimos variables categoricas a numericas
df_copy = pd.get_dummies(df_copy, dtype=int, drop_first=True)

In [34]:
#Separamos las variables a utilizar (Entrenamiento) (Predicción)
X = df_copy.drop('G3', axis=1) #Seleccionamos todas las variables menos G3
y = df_copy[['G3']] #Seleccionamos G3 la cual es la variable que queremos predecir 

In [36]:
#Realizamos la separacion de nuestros datos  20% test y 80% Entrenamiento 
X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=42, test_size=0.2)

#Ahora tenemos que escalar nuestros datos para que el modelo no se confunda 
#Variables a escalar debido que las variables categroicas ya están normalizadas
vars = ['age','Medu','Fedu','traveltime','studytime','failures','famrel','freetime','goout','Dalc','Walc','health','absences','G1','G2']

#Instanciamos nuestro escaler:  Formula z = (x - promedio) / std(Desviación estandar)
escalerx = StandardScaler()#Para X
escalery = StandardScaler()#Para Y

#Para X
X_train[vars] = escalerx.fit_transform(X_train[vars]) #Aprende cuales datos y los escala
X_test[vars] = escalerx.transform(X_test[vars]) #Al aprender de los datos solo necesitamos transformar 

#Se repite el mismo procedimiento pero con Y
Y_train = escalery.fit_transform(Y_train)
Y_test = escalery.transform(Y_test)